In [1]:
from herpetologist import check_type
from typeguard import typechecked

In [2]:
@check_type
def greeting(name: str):
    print(name)
    
greeting('foo') # no error
greeting(1) # error

foo


Exception: 1 must be a <class 'str'>

In [3]:
@typechecked
def greeting(name: str):
    pass

greeting('husein') # no error
greeting(1) # error

TypeError: type of argument "name" must be str; got int instead

In [4]:
from typing import List, Tuple, Dict

In [5]:
@check_type
def greeting(name: List[str]):
    pass

greeting(['h']) # no error
greeting([1]) # error

Exception: [1] must be a typing.List[str]

In [6]:
@typechecked
def greeting(name: List[str]):
    pass

greeting(['h']) # no error
greeting([1]) # error

TypeError: type of argument "name"[0] must be str; got int instead

In [7]:
@check_type
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # error

Exception: [('h', 1, 1)] must be a typing.List[typing.Tuple[str, int, float]]

In [8]:
@typechecked
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # no error, supposedly error

In [9]:
@check_type
def greeting(name: List[Tuple[str, int, Dict[int, int]]]):
    pass

greeting([('h', 1, {1:2})]) # no error
greeting([('h', 1, {1:2.2})]) # error

Exception: [('h', 1, {1: 2.2})] must be a typing.List[typing.Tuple[str, int, typing.Dict[int, int]]]

In [10]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]):
    pass

greeting((([{1: 'str'}],), ([1], [1]))) # no error
greeting((([{1: 'str'}],), ([1], [1.1]))) # error

Exception: (([{1: 'str'}],), ([1], [1.1])) must be a typing.Tuple[typing.Tuple[typing.List[typing.Dict[int, str]]], typing.Tuple[typing.List[int]]]

In [11]:
class ClassA:
    def __init__(self):
        pass

In [12]:
a = ClassA()

In [13]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, ClassA]]], Tuple[List[int], float, Tuple[Dict[str, str]]]]):
    pass

greeting((([{1: a}],), ([1], 1.1, ({'a': 'a'},)))) # no error
greeting((([{1: 'str'}],), ([1], 1.1, ({'a': 'a'}, {'a': 1})))) # error

Exception: (([{1: 'str'}],), ([1], 1.1, ({'a': 'a'}, {'a': 1}))) must be a typing.Tuple[typing.Tuple[typing.List[typing.Dict[int, __main__.ClassA]]], typing.Tuple[typing.List[int], float, typing.Tuple[typing.Dict[str, str]]]]